### UFC, CC, DC, BCC (2019.1), Mineração de Dados (CK0223), Prof.: J. Macedo

# Naive Bayes - Trabalho

**Equipe**
- Caio Rodrigues Costa (384345)

- Hélio Henrique Barbosa Rocha (396446)

- Vando Aderson Sacramento Alves (343177)

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

**Attributos**
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

**Classes**
1. unacc, acc, good, vgood

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import csv
import math
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
def isint(s):
    try:
        int(s)
    except:
        return False
    return True


def loadCsv(filename):
    lines = csv.reader(open(filename, 'r'))
    dataset = list(lines)
    for i in range(len(dataset)):
        for j in range(len(dataset[0])):
            if isint(dataset[i][j]):
                dataset[i][j] = float(dataset[i][j])
            else:
                if dataset[i][j] == 'vhigh' or dataset[i][j] == 'vgood':
                    dataset[i][j] = float(4)
                elif dataset[i][j] == 'high' or dataset[i][j] == 'big' \
                    or dataset[i][j] == 'good':
                    dataset[i][j] = float(3)
                elif dataset[i][j] == 'med' or dataset[i][j] == 'acc':
                    dataset[i][j] = float(2)
                elif dataset[i][j] == 'low' or dataset[i][j] == 'small' \
                    or dataset[i][j] == 'unacc':
                    dataset[i][j] = float(1)
                elif dataset[i][j] == '5more':
                    dataset[i][j] = float(5)
                elif dataset[i][j] == 'more':
                    dataset[i][j] = float(6)
    return dataset


In [ ]:
filename = 'carData.csv'
dataset = loadCsv(filename)


In [ ]:
cars = {
    'data': np.array([(dataset[i])[0:6] for i in range(len(dataset))]),
    'target': np.array([dataset[i][-1] for i in range(len(dataset))]),
    'DESCR': '',
    'feature_names': 'buying maint doors persons lug_boot safety',
    'data_filename': '',
    'target_filename': '',
}

cols = cars['feature_names'].split()
X, y = cars['data'], cars['target']
df = pd.DataFrame(X, columns=cols)


In [ ]:
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]


def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if vector[-1] not in separated:
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


def mean(numbers):
    return sum(numbers) / float(len(numbers))


def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) \
        / float(len(numbers) - 1)
    return math.sqrt(variance)


def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in
                 zip(*dataset)]
    del summaries[-1]
    return summaries


def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for (classValue, instances) in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries


def calculateProbability(x, mean, stdev):
    try:
        exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    except ZeroDivisionError:
        exponent = 0
        
    try:
        z = 1 / (math.sqrt(2 * math.pi) * stdev) * exponent 
    except ZeroDivisionError:
        z = 0
    
    return z


def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for (classValue, classSummaries) in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            (mean, stdev) = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities


def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    (bestLabel, bestProb) = (None, -1)
    for (classValue, probability) in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel


def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions


def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return correct / float(len(testSet)) * 100.0


In [ ]:
# criacao das variaveis de train e test, particionamento de dados]
splitRatio = 0.2
(X_train, X_test, y_train, y_test) = train_test_split(X, 
                                                      y, 
                                                      test_size=splitRatio)


## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
# Construcao do modelo Naive-Bayes Multinomial
classificadorMNB = MultinomialNB()

modelo = classificadorMNB.fit(X_train, y_train)
predicoes = classificadorMNB.predict(X_test)

result = modelo.score(X_test, y_test)

# Aplicando 'k-Fold Cross Validation'
acc = cross_val_score(estimator=classificadorMNB, 
                      X=X, 
                      y=y, 
                      cv=10)

print ('Acurácia Naive-Bayes Multinomial (sciKitLearn):')
print ('-------------------------------------------------')
print ('Score   \t| Cross-Validation')
print ('-------------------------------------------------')
print ('{:.2%}  \t| {:.2%} +/-{:.2%}'.format(result, acc.mean(), acc.std()))
print ('-------------------------------------------------')


Acurácia Naive-Bayes Multinomial (sciKitLearn):
-------------------------------------------------
Score   	| Cross-Validation
-------------------------------------------------
68.50%  	| 69.85% +/-0.35%
-------------------------------------------------


## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.

In [ ]:
def tset(X, y):
    t_set = [np.append(X[i], y[i]) for i in range(len(X))]
    
    return t_set


# mesmos dados sao utilizados para calculos nos modelos das questoes 1 e 2
trainingSet, testSet = tset(X_train, y_train), tset(X_test, y_test)


(trainingSet, testSet) = splitDataset(dataset, splitRatio)


In [ ]:
def naiveBayes():
    # preparar modelo
    summaries = summarizeByClass(trainingSet)

    # testar modelo
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print ('Acurácia Naive-Bayes Multinomial:')
    print ('-----------------------------------')
    print ('Score')
    print ('-----------------------------------')
    print ('{:.2f}%'.format(accuracy))
    print ('-----------------------------------')


naiveBayes()

Acurácia Naive-Bayes Multinomial:
-----------------------------------
Score
-----------------------------------
77.95%
-----------------------------------


### Discusão

Conforme resultados observados, pode-se considerar que os algoritmos empregados neste trabalho são eficazes. Para algumas situações, a despeito de determinados presupostos, a classificação mostra-se apropriada. Ainda assim, trata-se de um algoritmo altamente dependente da frequência relativa de cada classe. Dado que a probabilidade de verossimilhança é proporcional à probabilidade marginal das classes, aquelas que possuem uma frequência muito maior em relação as demais terão uma probabilidade maior, e impactarão no resultado da probabilidade final. Por isso, as classes com menores ocorrências acabam não tendo um desempenho satisfatório. Isto é ainda mais evidente no método implementado pelo sciKitLearn. Por utilizar diferentes métodos de contagem, ainda que possua um melhor custo computacional, ele acaba subestimando as probabilidades das classes menos frequentes e, por consequência, tem-se que as predições caem quase que exclusivamente nas classes maior frequência.